In [28]:
import os
os.environ['AIOD_RAIL_API_KEY'] = '9b9c7f0cd6af7656b2b480d842d79863'

In [29]:
import json
import aiod_rail_sdk
import aiod_rail_sdk.configuration
from client.client import RailClient


In [30]:
script_path = '/home/eddie/Projects/script.py'
requirements_path = '/home/eddie/Projects/requirements.txt'
image_path = '/home/eddie/Projects/Dockerfile'


We can either load template as whole json and create experiment from it

In [ ]:
with open('your-path-to-template-here') as f:
  template = json.load(f)

Or we can define our template in code and use paths defined earlier to pass script, requirements and image information to template when calling the method create_experiment_template

In [24]:
template = {
    "name": "MyExperimentNumberTwo",
    "description": "I created this on 30.5.2024",
    "task": "TEXT_CLASSIFICATION",
    "datasets_schema": {
        "cardinality": "1-1"
    },
    "models_schema": {
        "cardinality": "1-1"
    },
    "envs_required": [
        {
            "name": "SPLIT_NAME",
            "description": "name of a subset"
        }
    ],
    "envs_optional": [],
    "available_metrics": [
        "accuracy"
    ],
    "public": True
}

Create the client

In [46]:
config = aiod_rail_sdk.configuration.Configuration(host='http://localhost/api')
my_client = RailClient(config)
my_client.config.api_key

{'APIKeyHeader': '9b9c7f0cd6af7656b2b480d842d79863'}

Experiment Templates endpoints

Create experiment template from json file

In [10]:
resp = my_client.experiments_templates.create_experiment_template(template=(script_path, requirements_path, image_path, template))
resp

ExperimentTemplateResponse(name='MyExperimentNumberTwo', description='I created this on 30.5.2024', task=<TaskType.TEXT_CLASSIFICATION: 'TEXT_CLASSIFICATION'>, datasets_schema=AssetSchema(cardinality=<AssetCardinality.ENUM_1_MINUS_1: '1-1'>), models_schema=AssetSchema(cardinality=<AssetCardinality.ENUM_1_MINUS_1: '1-1'>), envs_required=[EnvironmentVarDef(name='SPLIT_NAME', description='name of a subset')], envs_optional=[], script='import os\n\nos.environ["HF_HOME"] = "."\n\nimport json\nimport logging\n\nimport numpy as np\nimport sklearn.metrics as m\nimport torch\nimport wandb\nfrom datasets import load_dataset\nfrom tqdm import tqdm\nfrom transformers import AutoModelForSequenceClassification, AutoTokenizer\n\n\ndef get_device():\n    return "cuda" if torch.cuda.is_available() else "cpu"\n\n\ndef wandb_check():\n    necessary_envs = [\n        "WANDB_API_KEY",\n        "WANDB_ENTITY",\n        "WANDB_PROJECT",\n        "WANDB_NAME",\n    ]\n    return set(necessary_envs).issubset(o

Update the experiment

In [28]:
template_changed = template.copy()
template_changed['name'] = 'MyExperimentNumberOneUpdated'
template_changed['description'] = 'I updated this experiment on 30.5.2024'
template_changed

{'name': 'MyExperimentNumberOneUpdated',
 'description': 'I updated this experiment on 30.5.2024',
 'task': 'TEXT_CLASSIFICATION',
 'datasets_schema': {'cardinality': '1-1'},
 'models_schema': {'cardinality': '1-1'},
 'envs_required': [{'name': 'SPLIT_NAME', 'description': 'name of a subset'}],
 'envs_optional': [],
 'available_metrics': ['accuracy'],
 'public': True}

Let's check the experiment templates, which are not approved and not built and take the one which we created from them to select it for update with template we defined above

In [27]:
experiment_templates = my_client.experiments_templates.get(finalized=False, approved=False)
experiment_template_to_update = None 
for experiment_template in experiment_templates:
    print(f'id: {experiment_template.id}\nname: {experiment_template.name}')
    print("-"*100)
    if experiment_template.name == 'MyExperimentNumberOne':
        experiment_template_to_update = experiment_template

id: 6655b12972f875012fe205f5
name: CreatedExperiment
----------------------------------------------------------------------------------------------------
id: 6655b24a72f875012fe205f6
name: ExperimentChanged
----------------------------------------------------------------------------------------------------
id: 665846af824d3eb3dc996092
name: MyExperimentNumberOne
----------------------------------------------------------------------------------------------------
id: 66584d10824d3eb3dc996093
name: MyExperimentNumberTwo
----------------------------------------------------------------------------------------------------


Let's update the selected experiment template, we will check the experiment id to verify that it's the same one we updated

In [29]:
updated_experiment_template = my_client.experiments_templates.update(experiment_template_to_update.id, template=(script_path, requirements_path, image_path, template_changed))
print(f'id: {updated_experiment_template.id},\nname: {updated_experiment_template.name}')


id: 665846af824d3eb3dc996092,
name: MyExperimentNumberOneUpdated


Let's visualize experiment templates which are approved and built 

In [47]:
experiment_templates = my_client.experiments_templates.get(finalized=True, approved=True)
for experiment_template in experiment_templates:
    print(f"id: {experiment_template.id} | name: {experiment_template.name} | approved: {experiment_template.approved}\n")

id: 6655ae4172f875012fe205f4 | name: ExpTempl | approved: True

id: 66570433b986f37c731ead6c | name: ExpTempl | approved: True



We can also get just their count if needed

In [40]:
my_client.experiments_templates.count(finalized=True, approved=True)

2

Let's get the first experiment template by it's id

In [48]:
experiment_template = my_client.experiments_templates.get_by_id(experiment_templates[0].id)
print(f"id: {experiment_template.id} | name: {experiment_template.name} | archived: {experiment_template.archived}\n")

id: 6655ae4172f875012fe205f4 | name: ExpTempl | archived: False



We will archive this experiment template

In [49]:
my_client.experiments_templates.archive(experiment_template.id, archived=True)

We can now verify that experiment template has been archived

In [50]:
experiment_template = my_client.experiments_templates.get_by_id(experiment_templates[0].id)
print(f"id: {experiment_template.id} | name: {experiment_template.name} | archived: {experiment_template.archived}\n")

id: 6655ae4172f875012fe205f4 | name: ExpTempl | archived: True



We can also view number of datasets we can browse

In [5]:
my_client.datasets.count()

411987

Let's get two of them

In [6]:
my_client.datasets.get(offset=0, limit=2)

[Dataset(platform='huggingface', platform_resource_identifier='acronym_identification', name='acronym_identification', date_published=datetime.datetime(2022, 3, 2, 23, 29, 22), same_as='https://huggingface.co/datasets/acronym_identification', is_accessible_for_free=True, version=None, issn=None, measurement_technique=None, temporal_coverage=None, ai_asset_identifier=2, ai_resource_identifier=2, aiod_entry=AIoDEntryRead(editor=[], status='published', date_modified=datetime.datetime(2023, 12, 19, 10, 25, 29), date_created=datetime.datetime(2023, 12, 19, 10, 25, 29)), alternate_name=[], application_area=[], citation=[], contact=[], creator=[], description=Text(plain='Acronym identification training and development sets for the acronym identification task at SDU@AAAI-21.', html=None), distribution=[Distribution(platform=None, platform_resource_identifier=None, checksum=None, checksum_algorithm=None, copyright=None, content_url='https://huggingface.co/datasets/acronym_identification/resolve

We can either load experiment description from file

In [ ]:
with open('your-path-to-experiment-config-json') as f:
    experiment = json.load(f)
experiment

Or we can define it by ourselves in code as dictionary, we have to specify an experiment template from which experiment will be created, let's pick the one from approved and built

In [5]:
experiment_templates = my_client.experiments_templates.get(finalized=True, approved=True)
for experiment_template in experiment_templates:
    print(f"id: {experiment_template.id} | name: {experiment_template.name} | approved: {experiment_template.approved}\n")


id: 6655ae4172f875012fe205f4 | name: ExpTempl | approved: True

id: 66570433b986f37c731ead6c | name: ExpTempl | approved: True



In [9]:
    
experiment = {
    "name": "MyNewExperiment",
    "description": "I created this experiment on 30.5.2024",
    "publication_ids": [],
    "experiment_template_id": experiment_templates[1].id,
    "dataset_ids": [
        "1"
    ],
    "model_ids": [
        "2"
    ],
    "env_vars": [
        {
            "key": "SPLIT_NAME",
            "value": "train"
        }
    ],
    "public": True
}
experiment

{'name': 'MyNewExperiment',
 'description': 'I created this experiment on 30.5.2024',
 'publication_ids': [],
 'experiment_template_id': '66570433b986f37c731ead6c',
 'dataset_ids': ['1'],
 'model_ids': ['2'],
 'env_vars': [{'key': 'SPLIT_NAME', 'value': 'train'}],
 'public': True}

Create experiment from provided json file

In [11]:
experiment = my_client.experiments.create_experiment(experiment=experiment)
experiment

ExperimentResponse(name='MyNewExperiment', description='I created this experiment on 30.5.2024', experiment_template_id='66570433b986f37c731ead6c', publication_ids=[], dataset_ids=['1'], model_ids=['2'], env_vars=[EnvironmentVar(key='SPLIT_NAME', value='train')], public=True, id='665864354861eda4a581de86', created_at=datetime.datetime(2024, 5, 30, 11, 34, 13, 449350, tzinfo=TzInfo(UTC)), updated_at=datetime.datetime(2024, 5, 30, 11, 34, 13, 449344, tzinfo=TzInfo(UTC)), archived=False, mine=True)

Let's display all experiments, we can see our experiment TodayExperiment as the last listed

In [32]:
experiments = my_client.experiments.get()
for e in experiments:
    print(f'id: {e.id} | name: {e.name} | archived: {e.archived}')

id: 6655d24f72f875012fe205f9 | name: Exp01 | archived: False
id: 6655d51372f875012fe205fa | name: Exp01 | archived: True
id: 6657103fb986f37c731ead6d | name: TodayExperiment | archived: False
id: 6658634d4861eda4a581de85 | name: MyNewExperiment | archived: False


As we can see from output above, one of the experiments is archived, we can use count to display how many of experiments are archived 

In [19]:
count = my_client.experiments.count(archived=True)
count


1

We can now run the experiment which we will select by providing it's id, lets select the one which we created before 

In [33]:
experiment_for_run = [e for e in experiments if e.name == "MyNewExperiment"][0]
experiment_for_run

ExperimentResponse(name='MyNewExperiment', description='I created this experiment on 30.5.2024', experiment_template_id='66570433b986f37c731ead6c', publication_ids=[], dataset_ids=['1'], model_ids=['2'], env_vars=[EnvironmentVar(key='SPLIT_NAME', value='train')], public=True, id='6658634d4861eda4a581de85', created_at=datetime.datetime(2024, 5, 30, 11, 30, 20, 823000), updated_at=datetime.datetime(2024, 5, 30, 11, 30, 20, 823000), archived=False, mine=True)

In [20]:
run = my_client.experiments.run_experiment(id=experiment_for_run.id)
run

ExperimentRunResponse(id='665865f34861eda4a581de87', created_at=datetime.datetime(2024, 5, 30, 11, 41, 39, 427878, tzinfo=TzInfo(UTC)), updated_at=datetime.datetime(2024, 5, 30, 11, 41, 39, 427882, tzinfo=TzInfo(UTC)), retry_count=0, state=<RunState.CREATED: 'CREATED'>, metrics={}, archived=False, public=True, mine=True, experiment_id='6658634d4861eda4a581de85')

We can display number of experiment runs and it's count

In [35]:
running = my_client.experiments.get_experiments_run_count(id=experiment_for_run.id)
running

1

In [36]:
experiment_runs = my_client.experiments.get_experiments_run(id=experiment_for_run.id)
for e_r in experiment_runs:
    print(f"{e_r}")

id='665865f34861eda4a581de87' created_at=datetime.datetime(2024, 5, 30, 11, 41, 39, 427000) updated_at=datetime.datetime(2024, 5, 30, 11, 42, 54, 625000) retry_count=0 state=<RunState.FINISHED: 'FINISHED'> metrics={} archived=False public=True mine=True experiment_id='6658634d4861eda4a581de85'


Let's display logs from the run which we executed on the experiment

In [55]:
logs = my_client.experiments.logs_experiment_run(id=experiment_runs[0].id)

from pprint import pprint

pprint(logs)

('{"workflow_logs": "2024-05-30 11:41:59,891 | root | MainThread | INFO | '
 'Publishing step:0, cmd: set -a && source .env && set +a && python script.py, '
 'total steps 1 to MQ\\n2024-05-30 11:42:51,039 | root | MainThread | INFO | '
 'Workflow e420a9bd-81d8-437d-824f-813328d196df finished. Files available at '
 '/var/reana/users/00000000-0000-0000-0000-000000000000/workflows/e420a9bd-81d8-437d-824f-813328d196df.\\n\\n", '
 '"job_logs": {"18d8b2cf-8a55-4e05-8f01-66f9677cd620": {"workflow_uuid": '
 '"e420a9bd-81d8-437d-824f-813328d196df", "job_name": "Execute Python script", '
 '"compute_backend": "Kubernetes", "backend_job_id": '
 '"reana-run-job-588e8a65-4b22-4271-add3-94a118c7107e", "docker_img": '
 '"docker.io/marveso/rail-exp-templates:template-66570433b986f37c731ead6c", '
 '"cmd": "set -a && source .env && set +a && python script.py", "status": '
 '"finished", "logs": "job: :\\n '
 '/usr/local/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: '
 'FutureWarning: 

We can also download files from experiment, this call will download script.py into the folder where this code is located

In [38]:
my_client.experiments.download_experiment_run(id=experiment_runs[0].id, filepath='script.py', to_dir='run_out')

Let's archive our experiment which we ran earlier

In [39]:
my_client.experiments.archive(id=experiment_for_run.id, archived=True)

As we can see from output, our experiment MyNewExperiment has been archived

In [40]:
archived_experiments = my_client.experiments.get(archived=True)

for a_e in archived_experiments:
    print(f"id: {a_e.id} | name: {a_e.name} | archived: {a_e.archived}")

id: 6655d51372f875012fe205fa | name: Exp01 | archived: True
id: 6658634d4861eda4a581de85 | name: MyNewExperiment | archived: True


We can also delete the run which we executed earlier but before that, we have to un-archive the experiment to be able to delete it

In [52]:
unarchived_experiment = my_client.experiments.archive(id=experiment_for_run.id, archived=False)

In [56]:
my_client.experiments.delete_experiment_run(id=experiment_runs[0].id)

As we can see the run is deleted

In [57]:
count = my_client.experiments.get_experiments_run_count(id=experiment_for_run.id)
count

0